In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('../input/tabular-playground-series-jul-2021/train.csv')

In [ ]:
sample = pd.read_csv('../input/tabular-playground-series-jul-2021/sample_submission.csv')

In [ ]:
df.head()

In [ ]:
sample.head()

In [ ]:
test = pd.read_csv('../input/tabular-playground-series-jul-2021/test.csv')

In [ ]:
test.head()

In [ ]:
df.shape

In [ ]:
df['date_time'].head(30)

In [ ]:
df['date_time'][30:40]

In [ ]:
(df['date_time'].shape[0]-6)/24

In [ ]:
#data consists reading of each hour of around 296 days

In [ ]:
df['date_time'].tail()

In [ ]:
sns.displot(df['target_benzene'])

In [ ]:
plt.figure(figsize = (12,7))
plt.subplot(1,2,1)
temp = plt.hist(df['target_carbon_monoxide'],bins = 100)
plt.xlabel('target carbon monoxide')
plt.ylabel('frequency')
plt.subplot(1,2,2)
temp = plt.hist(df['target_nitrogen_oxides'],bins = 100)
plt.xlabel('target nitrogen oxides')
plt.ylabel('frequency')

In [ ]:
len(df['target_carbon_monoxide'].unique())

In [ ]:
# Carbon monoxide can be considered a categorical variable

In [ ]:
x = range(0,len(df['date_time']))
plt.plot(x,df['target_benzene'])

In [ ]:
plt.figure(figsize = (12,10))
plt.subplot(2,1,1)
plt.plot(x,df['target_nitrogen_oxides'])
plt.subplot(2,1,2)
plt.plot(x,df['target_carbon_monoxide'])

In [ ]:
# Data very irregular, time series prediction approach not possible.

In [ ]:
data = df.drop(columns = ['date_time'])

In [ ]:
print(data.columns)

In [ ]:
sns.displot(data['deg_C'])

In [ ]:
# Normal Distributed

In [ ]:
plt.figure(figsize = (14,21))
plt.subplot(4,2,1)
temp = plt.hist(data['sensor_1'],bins = 100)
plt.xlabel('sensor 1 reading')
plt.ylabel('frequency')
plt.subplot(4,2,2)
temp = plt.hist(data['sensor_2'],bins = 100)
plt.xlabel('sensor 2 reading')
plt.ylabel('frequency')
plt.subplot(4,2,3)
temp = plt.hist(data['sensor_3'],bins = 100)
plt.xlabel('sensor 3 reading')
plt.ylabel('frequency')
plt.subplot(4,2,4)
temp = plt.hist(data['sensor_4'],bins = 100)
plt.xlabel('sensor 4 reading')
plt.ylabel('frequency')
plt.subplot(4,2,5)
temp = plt.hist(data['sensor_5'],bins = 100)
plt.xlabel('sensor 5 reading')
plt.ylabel('frequency')
plt.subplot(4,2,6)
temp = plt.hist(data['relative_humidity'],bins = 100)
plt.xlabel('Relative Humidity')
plt.ylabel('frequency')
plt.subplot(4,2,7)
temp = plt.hist(data['absolute_humidity'],bins = 100)
plt.xlabel('Absolute Humidity')
plt.ylabel('frequency')


In [ ]:
labels = data[['target_benzene','target_carbon_monoxide','target_nitrogen_oxides']]

In [ ]:
data.drop(columns =['target_benzene','target_carbon_monoxide','target_nitrogen_oxides'],inplace = True )

In [ ]:
from sklearn.preprocessing import Normalizer

In [ ]:
n = Normalizer()

In [ ]:
n.fit(data)

In [ ]:
data_normal = n.transform(data)

In [ ]:
data_normal = pd.DataFrame(data_normal)

In [ ]:
data_normal.columns = data.columns

In [ ]:
data_normal.head()

In [ ]:
import tensorflow.keras as keras

In [ ]:
temp = data_normal
temp['target_carbon_monoxide'] = labels['target_carbon_monoxide']
temp['target_benzene'] = labels['target_benzene']
temp['target_nitrogen_oxides'] = labels['target_nitrogen_oxides']

In [ ]:
data_normal.head()


In [ ]:
from sklearn.model_selection import train_test_split
train,val = train_test_split(temp,test_size = 0.2)

In [ ]:
trainx = train[['deg_C', 'relative_humidity', 'absolute_humidity', 'sensor_1',
       'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5']]
trainy = train[['target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides']]
valx = val[['deg_C', 'relative_humidity', 'absolute_humidity', 'sensor_1',
       'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5']]
valy = val[['target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides']]

In [ ]:
import tensorflow as tf
import math
from keras import backend as K

In [ ]:
keras.backend.clear_session()
def scheduler(epoch, lr):
#     if epoch < 10:
#         return lr
#     else:
        return lr * tf.math.exp(-0.01)
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
model = keras.models.Sequential([
    keras.layers.Dense(6,input_dim = 8,activation= 'relu'),
    keras.layers.Dense(4,activation= 'relu'),
    keras.layers.Dense(3,activation= 'relu'),
])
model.compile(optimizer = tf.keras.optimizers.SGD(learning_rate = 10),loss  =tf.keras.losses.MeanSquaredLogarithmicError(),metrics = ['mse'])
history = model.fit(trainx,trainy,callbacks = [callback],epochs = 500,verbose =0)

In [ ]:
plt.semilogx(history.history['lr'],history.history['loss'])


In [ ]:
keras.backend.clear_session()
early = keras.callbacks.EarlyStopping(patience = 20)
model_check = keras.callbacks.ModelCheckpoint('model.h5',save_best_only = True)
model = keras.models.Sequential([
    keras.layers.Dense(6,input_dim = 8,activation= 'relu'),
#     keras.layers.Dropout(0.5),
    keras.layers.Dense(5,activation= 'relu'),
#     keras.layers.Dropout(0.5),
    keras.layers.Dense(3,activation= 'relu')
])
model.compile(loss  =tf.keras.losses.MeanSquaredLogarithmicError(),optimizer =  'adam',metrics = ['mse'])

In [ ]:
history = model.fit(trainx,trainy,callbacks = [model_check,early],verbose = 0,epochs = 500,validation_data = (valx,valy))

In [ ]:
plt.figure(figsize = (12,7))
plt.subplot(2,1,1)
plt.plot(range(1,501),history.history['loss'])
plt.plot(range(1,501),history.history['val_loss'])
plt.title('Validation and Train loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.subplot(2,1,2)
plt.plot(range(1,501),history.history['mse'])
plt.plot(range(1,501),history.history['val_mse'])
plt.title('Validation and Train mse')
plt.xlabel('epochs')
plt.ylabel('mse')

In [ ]:
model = keras.models.load_model('model.h5')

In [ ]:
test.head()

In [ ]:
test = test.drop(columns = ['date_time'])

In [ ]:
test = n.transform(test)

In [ ]:
preditions = model.predict(test)

In [ ]:
sample.head()

In [ ]:
predictions= pd.DataFrame(preditions)

In [ ]:
predictions.head()

In [ ]:
sample['target_carbon_monoxide'] = predictions[0]

In [ ]:
sample['target_benzene'] = predictions[1]

In [ ]:
sample['target_nitrogen_oxides'] = predictions[2]

In [ ]:
sample.to_csv('submit5.csv',index = False)

In [ ]:
test = pd.read_csv('../input/tabular-playground-series-jul-2021/test.csv')

In [ ]:
df.head()

# Work In Progress! Trying to improve from the score of 0.37. If you like my work please give your feedback! Upadate coming very soon.